In [4]:
import os
from posixpath import join
import paramiko
from scp import SCPClient
import subprocess

run_id = 19
jar_name = 'letter-frequency-1.0-SNAPSHOT.jar'
# connect with ssh


First analyze the letter frequencies in books across the eras. As performance is not relevant now, the job will be executed with 1 reducer, using combiner.
We hypotize that the input files are already in the input folder in hdfs, in /user/hadoop/letter_analysis/input/.

In [5]:
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect('10.1.1.124', username='hadoop', password='ubuntu')
root_dir = '/user/hadoop/historical_analysis'

# run the job for each file
for txt_file in os.listdir('../resources/historical_analysis/input'):
    print(f'Processing {txt_file}')
    input_dir = join(root_dir, 'input', txt_file)
    output_dir = join(root_dir, f'output_{run_id}', txt_file.split('.')[0])
    stdin, stdout, stderr = ssh.exec_command(f'/opt/hadoop/bin/hadoop jar {jar_name} it.unipi.cloud.MapReduceApp {input_dir} {output_dir}/count {output_dir}/freq 1 inmappercombiner')
    print(stderr.read().decode('utf-8'))
    print(stdout.read().decode('utf-8'))

# copy the output to local
stdin, stdout, stderr = ssh.exec_command(f'/opt/hadoop/bin/hadoop fs -copyToLocal /user/hadoop/historical_analysis/output_{run_id} .')
print(stderr.read().decode('utf-8'))

if not os.path.exists(f'../resources/historical_analysis/output_{run_id}'):
    os.mkdir(f'../resources/historical_analysis/output_{run_id}')
    
scp = SCPClient(ssh.get_transport())
scp.get(f'output_{run_id}', '../resources/historical_analysis/', recursive=True)
scp.close()
ssh.close()

Processing 1294.txt
2024-06-18 08:39:46,560 INFO client.RMProxy: Connecting to ResourceManager at hadoop-namenode/10.1.1.124:8032
2024-06-18 08:39:47,102 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/hadoop/.staging/job_1716534983775_0173
2024-06-18 08:39:47,201 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2024-06-18 08:39:47,360 WARN hdfs.DataStreamer: Caught exception
java.lang.InterruptedException
	at java.lang.Object.wait(Native Method)
	at java.lang.Thread.join(Thread.java:1257)
	at java.lang.Thread.join(Thread.java:1331)
	at org.apache.hadoop.hdfs.DataStreamer.closeResponder(DataStreamer.java:986)
	at org.apache.hadoop.hdfs.DataStreamer.endBlock(DataStreamer.java:640)
	at org.apache.hadoop.hdfs.DataStreamer.run(DataStreamer.java:810)
2024-06-18 08:39:47,420 INFO input.FileInputFormat: Total input files to process : 1
2024-06-18 08:39:47,471 INFO sasl.SaslDataTra

After that we can analyze the output files.

In [6]:
import os
import pandas as pd

import matplotlib.pyplot as plt

output_folder = '/user/hadoop/historical_analysis/output'

# Get the list of files in the output folder
files = os.listdir(output_folder)

letter_over_years = []

# Process each file
for file in files:
    # Extract the year from the file name
    year = file.split('_')[1].split('.')[0]
    
    # Read the file as a pandas DataFrame
    df = pd.read_csv(os.path.join(output_folder, file), names=['Letter', 'Frequency'])
    letter_over_years.append((year, df['Frequency'].where(df['Letter'] == 'a')))
    
# Plot the trend of letter frequencies over time
fig, ax = plt.subplots(figsize=(10, 6))

for year, df in letter_over_years.items():
    ax.plot(df['Letter'], df['Relative Frequency'], label=year)

ax.set_xlabel('Year')
ax.set_ylabel('Relative Frequency')
ax.set_title('Trend of letter \'a\' frequency over time')
ax.legend()
plt.show()

FileNotFoundError: [WinError 3] Impossibile trovare il percorso specificato: '/user/hadoop/historical_analysis/output'